In [2]:
import torch
import torch.nn as nn
from audiocraft.modules.conv import *
from audiocraft.modules.lstm import *
from audiocraft.modules.seanet import SEANetResnetBlock2d, SEANetResnetBlock
import typing as tp
import numpy as np


class Printer(nn.Module):

    def __init__(self, cls):
        super().__init__()
        self.cls = str(cls)
        self.id = nn.Identity()
    
    def forward(self, x):
        print(f"{self.cls}: {x.shape}")
        x = self.id(x)
        return x


class SEANetEncoder2d(nn.Module):
    """SEANet encoder.

    Args:
        channels (int): Audio channels.
        dimension (int): Intermediate representation dimension.
        n_filters (int): Base width for the model.
        n_residual_layers (int): nb of residual layers.
        ratios (Sequence[int]): kernel size and stride ratios. The encoder uses downsampling ratios instead of
            upsampling ratios, hence it will use the ratios in the reverse order to the ones specified here
            that must match the decoder order. We use the decoder order as some models may only employ the decoder.
        activation (str): Activation function.
        activation_params (dict): Parameters to provide to the activation function.
        norm (str): Normalization method.
        norm_params (dict): Parameters to provide to the underlying normalization used along with the convolution.
        kernel_size (int): Kernel size for the initial convolution.
        last_kernel_size (int): Kernel size for the initial convolution.
        residual_kernel_size (int): Kernel size for the residual layers.
        dilation_base (int): How much to increase the dilation with each layer.
        causal (bool): Whether to use fully causal convolution.
        pad_mode (str): Padding mode for the convolutions.
        true_skip (bool): Whether to use true skip connection or a simple
            (streamable) convolution as the skip connection in the residual network blocks.
        compress (int): Reduced dimensionality in residual branches (from Demucs v3).
        lstm (int): Number of LSTM layers at the end of the encoder.
        disable_norm_outer_blocks (int): Number of blocks for which we don't apply norm.
            For the encoder, it corresponds to the N first blocks.
    """
    def __init__(self, channels: int = 1, dimension: int = 128, n_filters: int = 32, n_residual_layers: int = 3,
                 ratios: tp.List[int] = [8, 5, 4, 2], activation: str = 'ELU', activation_params: dict = {'alpha': 1.0},
                 norm: str = 'none', norm_params: tp.Dict[str, tp.Any] = {}, kernel_size: int = 7,
                 last_kernel_size: int = 7, residual_kernel_size: int = 3, dilation_base: int = 2, causal: bool = False,
                 pad_mode: str = 'reflect', true_skip: bool = True, compress: int = 2, lstm: int = 0,
                 disable_norm_outer_blocks: int = 0, 
                 frequency_bins: int = 512, 
                 num_1x1_convolutions: int = 2,
                 temporal_ratios: tp.List[int] = [2]):
        super().__init__()
        self.num_1x1_convolutions = num_1x1_convolutions
        self.temporal_ratios = temporal_ratios
        self.channels = channels
        self.dimension = dimension
        self.n_filters = n_filters
        self.frequency_bins = frequency_bins
        assert self.frequency_bins == 512, "model handles 512 frequency channels, change manually and in config file" 
        self.ratios = list(reversed(ratios))
        del ratios
        self.n_residual_layers = n_residual_layers
        self.hop_length = np.prod(self.ratios)
        self.n_blocks = len(self.ratios) + 2  # first and last conv + residual blocks
        self.disable_norm_outer_blocks = disable_norm_outer_blocks
        assert self.disable_norm_outer_blocks >= 0 and self.disable_norm_outer_blocks <= self.n_blocks, \
            "Number of blocks for which to disable norm is invalid." \
            "It should be lower or equal to the actual number of blocks in the network and greater or equal to 0."

        act = getattr(nn, activation)

        chnls = n_filters
        freq_bins = frequency_bins

        model: tp.List[nn.Module] = [
            StreamableConv2d(2, chnls, kernel_size,
                             norm='none' if self.disable_norm_outer_blocks >= 1 else norm,
                             norm_kwargs=norm_params, causal=causal, pad_mode=pad_mode)
        ]
        # Take strides accross frequency dimension
        for i, ratio in enumerate(self.ratios):
            block_norm = 'none' if self.disable_norm_outer_blocks >= i + 2 else norm
            # Add residual layers
            for j in range(n_residual_layers):
                model += [
                    SEANetResnetBlock2d(chnls, kernel_sizes=[residual_kernel_size, 1],
                                        dilations=[dilation_base ** j, 1],
                                        norm=block_norm, norm_params=norm_params,
                                        activation=activation, activation_params=activation_params,
                                        causal=causal, pad_mode=pad_mode, compress=compress, true_skip=true_skip)]

            # Add downsampling layers
            model += [
                act(**activation_params),
                StreamableConv2d(chnls, chnls * 2,
                                 kernel_size=ratio * 2, stride=ratio,
                                 norm=block_norm, norm_kwargs=norm_params,
                                 causal=causal, pad_mode=pad_mode),
            ]
            chnls *= 2
            freq_bins //= 2

        # decrease channels using 1x1 convolutions
        for _ in range(self.num_1x1_convolutions):
            for _ in range(n_residual_layers):
                model += [
                    SEANetResnetBlock2d(chnls, kernel_sizes=[1, 1],
                                        dilations=[1, 1],
                                        norm=block_norm, norm_params=norm_params,
                                        activation=activation, activation_params=activation_params,
                                        causal=causal, pad_mode=pad_mode, compress=compress, true_skip=true_skip)]

            # Add downsampling layers
            model += [
                act(**activation_params),
                StreamableConv2d(chnls, chnls // 2,
                                 kernel_size=1, stride=1,
                                 norm=block_norm, norm_kwargs=norm_params,
                                 causal=causal, pad_mode=pad_mode),
            ]
            chnls //= 2

        # flatten: results in output shape of (B, chnsl * freq_bins, T)
        model += [nn.Flatten(1, 2)]
        flattened_dim = chnls * freq_bins

        # force desired latent size
        model += [
            act(**activation_params),
            StreamableConv1d(flattened_dim, dimension, 1,
                             norm='none' if self.disable_norm_outer_blocks == self.n_blocks else norm,
                             norm_kwargs=norm_params, causal=causal, pad_mode=pad_mode)
        ]

        # take temporal strides
        if len(self.temporal_ratios) > 0:
            for i, ratio in enumerate(self.temporal_ratios):
                block_norm = 'none' if self.disable_norm_outer_blocks >= i + 2 else norm
                # Add residual layers
                for j in range(n_residual_layers):
                    model += [
                        SEANetResnetBlock(dimension, kernel_sizes=[3, 1],
                                        dilations=[1, 1],
                                        norm=block_norm, norm_params=norm_params,
                                        activation=activation, activation_params=activation_params,
                                        causal=causal, pad_mode=pad_mode, compress=compress, true_skip=true_skip)]

                # Add downsampling layers
                model += [
                    act(**activation_params),
                    StreamableConv1d(dimension,dimension,
                                    kernel_size=ratio * 2, stride=ratio,
                                    norm=block_norm, norm_kwargs=norm_params,
                                    causal=causal, pad_mode=pad_mode),
                ]

        if lstm:
            model += [StreamableLSTM(dimension, num_layers=lstm)]

        model += [
            act(**activation_params),
            StreamableConv1d(dimension, dimension, last_kernel_size,
                             norm='none' if self.disable_norm_outer_blocks == self.n_blocks else norm,
                             norm_kwargs=norm_params, causal=causal, pad_mode=pad_mode)
        ]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)


class SEANetDecoder2d(nn.Module):
    """SEANet decoder.

    Args:
        channels (int): Audio channels.
        dimension (int): Intermediate representation dimension.
        n_filters (int): Base width for the model.
        n_residual_layers (int): nb of residual layers.
        ratios (Sequence[int]): kernel size and stride ratios.
        activation (str): Activation function.
        activation_params (dict): Parameters to provide to the activation function.
        final_activation (str): Final activation function after all convolutions.
        final_activation_params (dict): Parameters to provide to the activation function.
        norm (str): Normalization method.
        norm_params (dict): Parameters to provide to the underlying normalization used along with the convolution.
        kernel_size (int): Kernel size for the initial convolution.
        last_kernel_size (int): Kernel size for the initial convolution.
        residual_kernel_size (int): Kernel size for the residual layers.
        dilation_base (int): How much to increase the dilation with each layer.
        causal (bool): Whether to use fully causal convolution.
        pad_mode (str): Padding mode for the convolutions.
        true_skip (bool): Whether to use true skip connection or a simple.
            (streamable) convolution as the skip connection in the residual network blocks.
        compress (int): Reduced dimensionality in residual branches (from Demucs v3).
        lstm (int): Number of LSTM layers at the end of the encoder.
        disable_norm_outer_blocks (int): Number of blocks for which we don't apply norm.
            For the decoder, it corresponds to the N last blocks.
        trim_right_ratio (float): Ratio for trimming at the right of the transposed convolution under the causal setup.
            If equal to 1.0, it means that all the trimming is done at the right.
    """
    def __init__(self, channels: int = 1, dimension: int = 128, n_filters: int = 32, n_residual_layers: int = 3,
                 ratios: tp.List[int] = [8, 5, 4, 2], activation: str = 'ELU', activation_params: dict = {'alpha': 1.0},
                 final_activation: tp.Optional[str] = None, final_activation_params: tp.Optional[dict] = None,
                 norm: str = 'none', norm_params: tp.Dict[str, tp.Any] = {}, kernel_size: int = 7,
                 last_kernel_size: int = 7, residual_kernel_size: int = 3, dilation_base: int = 2, causal: bool = False,
                 pad_mode: str = 'reflect', true_skip: bool = True, compress: int = 2, lstm: int = 0,
                 disable_norm_outer_blocks: int = 0, trim_right_ratio: float = 1.0, frequency_bins: int = 512, 
                 num_1x1_convolutions: int = 2,
                 temporal_ratios: tp.List[int] = [2]):
        super().__init__()
        self.num_1x1_convolutions = num_1x1_convolutions
        self.temporal_ratios = temporal_ratios
        self.dimension = dimension
        self.channels = channels
        self.n_filters = n_filters
        self.frequency_bins = frequency_bins
        assert self.frequency_bins == 512, "model handles 512 frequency channels, change manually and in config file" 
        self.ratios = ratios
        del ratios
        self.n_residual_layers = n_residual_layers
        self.hop_length = np.prod(self.ratios)
        self.n_blocks = len(self.ratios) + 2  # first and last conv + residual blocks
        self.disable_norm_outer_blocks = disable_norm_outer_blocks
        assert self.disable_norm_outer_blocks >= 0 and self.disable_norm_outer_blocks <= self.n_blocks, \
            "Number of blocks for which to disable norm is invalid." \
            "It should be lower or equal to the actual number of blocks in the network and greater or equal to 0."

        act = getattr(nn, activation)
        mult = int(2 ** len(self.ratios))
        model: tp.List[nn.Module] = [
            StreamableConv1d(dimension, dimension, kernel_size,
                             norm='none' if self.disable_norm_outer_blocks == self.n_blocks else norm,
                             norm_kwargs=norm_params, causal=causal, pad_mode=pad_mode)
        ]
                

        if lstm:
            model += [StreamableLSTM(dimension, num_layers=lstm)]

        # reverse temporal strides 
        if len(temporal_ratios) > 0:
            for i, ratio in enumerate(self.temporal_ratios):
                block_norm = 'none' if self.disable_norm_outer_blocks >= self.n_blocks - (i + 1) else norm
                # Add upsampling layers
                model += [
                    act(**activation_params),
                    StreamableConvTranspose1d(dimension, dimension,
                                            kernel_size=ratio * 2, stride=ratio,
                                            norm=block_norm, norm_kwargs=norm_params,
                                            causal=causal, trim_right_ratio=trim_right_ratio),
                ]
                # Add residual layers
                for _ in range(n_residual_layers):
                    model += [
                        SEANetResnetBlock(dimension, kernel_sizes=[3, 1],
                                        dilations=[1, 1],
                                        activation=activation, activation_params=activation_params,
                                        norm=block_norm, norm_params=norm_params, causal=causal,
                                        pad_mode=pad_mode, compress=compress, true_skip=true_skip)]

        chnls = n_filters * (2 ** (len(self.ratios) - self.num_1x1_convolutions))
        freq_bins = frequency_bins // np.product(self.ratios)
        

        model += [
            StreamableConv1d(dimension, chnls * freq_bins, kernel_size,
                             norm='none' if self.disable_norm_outer_blocks == self.n_blocks else norm,
                             norm_kwargs=norm_params, causal=causal, pad_mode=pad_mode)
        ]
        model += [nn.Unflatten(1, (int(chnls), int(freq_bins)))]

        # scale channels using 1x1 convolutions
        for _ in range(self.num_1x1_convolutions):
            for _ in range(n_residual_layers):
                model += [
                    SEANetResnetBlock2d(chnls, kernel_sizes=[1, 1],
                                        dilations=[1, 1],
                                        norm=block_norm, norm_params=norm_params,
                                        activation=activation, activation_params=activation_params,
                                        causal=causal, pad_mode=pad_mode, compress=compress, true_skip=true_skip)]

            # Add downsampling layers
            model += [
                act(**activation_params),
                StreamableConv2d(chnls, chnls * 2,
                                 kernel_size=1, stride=1,
                                 norm=block_norm, norm_kwargs=norm_params,
                                 causal=causal, pad_mode=pad_mode),
            ]
            chnls *= 2

        # Upsample to raw audio scale
        for i, ratio in enumerate(self.ratios):
            block_norm = 'none' if self.disable_norm_outer_blocks >= self.n_blocks - (i + 1) else norm
            # Add upsampling layers
            model += [
                act(**activation_params),
                StreamableConvTranspose2d(chnls, chnls // 2,
                                          kernel_size=ratio * 2, stride=ratio,
                                          norm=block_norm, norm_kwargs=norm_params,
                                          causal=causal, trim_right_ratio=trim_right_ratio),
            ]
            # Add residual layers
            for j in range(n_residual_layers):
                model += [
                    SEANetResnetBlock2d(chnls // 2, kernel_sizes=[residual_kernel_size, 1],
                                        dilations=[dilation_base ** j, 1],
                                        activation=activation, activation_params=activation_params,
                                        norm=block_norm, norm_params=norm_params, causal=causal,
                                        pad_mode=pad_mode, compress=compress, true_skip=true_skip)]

            chnls //= 2

        # Add final layers
        model += [
            act(**activation_params),
            StreamableConv2d(n_filters, 2, last_kernel_size,
                             norm='none' if self.disable_norm_outer_blocks >= 1 else norm,
                             norm_kwargs=norm_params, causal=causal, pad_mode=pad_mode)
        ]
        # Add optional final activation to decoder (eg. tanh)
        if final_activation is not None:
            final_act = getattr(nn, final_activation)
            final_activation_params = final_activation_params or {}
            model += [
                final_act(**final_activation_params)
            ]
        # new_model = list()
        # for m in model:
        #     new_model.append(m)
        #     new_model.append(Printer(type(m)))
        self.model = nn.Sequential(*model)

    def forward(self, z):
        y = self.model(z)
        return y

In [4]:
import torch

x = torch.randn((2, 2, 512, 64))
print("Encoder:")
n_filt = 96
n_res = 3
encoder = SEANetEncoder2d(ratios=[2,2,2,2], n_filters=n_filt, lstm=2, n_residual_layers=n_res)
z = encoder(x)
print("\nLatent:")
print(z.shape)
print("\nDecoder")
decoder = SEANetDecoder2d(ratios=[2,2,2,2], n_filters=n_filt, lstm=2, n_residual_layers=n_res)
y = decoder(z)
print("\nOutput")
print(y.shape)

def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total trainable parameters: {total_params:,}")
    return total_params

dec_params = count_parameters(decoder)
enc_params = count_parameters(encoder)
print(f"tot: {enc_params + dec_params:,}")

Encoder:

Latent:
torch.Size([2, 128, 32])

Decoder

Output
torch.Size([2, 2, 512, 64])
Total trainable parameters: 52,095,026
Total trainable parameters: 49,286,768
tot: 101,381,794
